In [1]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

In [2]:
# Load data set for bugs and sentiments

print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)

C:\Users\asif\Desktop\bugs
-----------------
['Bugzilla_blocker_626.json', 'Bugzilla_critical_326.json', 'Bugzilla_major_660.json', 'Bugzilla_minor_1049.json', 'bugzilla_normal_5753.json', 'Bugzilla_trivial_682.json', 'Core_blocker_1618.json', 'Core_critical_10000.json', 'Core_major_7523.json', 'Core_minor_3105.json', 'Core_normal_10000.json', 'Core_trivial_1484.json', 'Firefox_blocker_223.json', 'Firefox_critical_596.json', 'Firefox_major_1205.json', 'Firefox_minor_965.json', 'Firefox_normal_10000.json', 'Firefox_trivial_681.json', 'Thunderbird_blocker_204.json', 'Thunderbird_critical_360.json', 'Thunderbird_major_583.json', 'Thunderbird_minor_476.json', 'Thunderbird_normal_8215.json', 'Thunderbird_trivial_242.json']


In [113]:
bug_reports=[]

for count, report in enumerate(BUG_REPORTS_LIST):
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            bug_reports.extend(temp_data["bugs"])
    

Test Set: Bugzilla_blocker_626.json
File:Bugzilla_blocker_626.json <> Reports: 627
Test Set: Bugzilla_critical_326.json
File:Bugzilla_critical_326.json <> Reports: 327
Test Set: Bugzilla_major_660.json
File:Bugzilla_major_660.json <> Reports: 661
Test Set: Bugzilla_minor_1049.json
File:Bugzilla_minor_1049.json <> Reports: 1050
Test Set: bugzilla_normal_5753.json
File:bugzilla_normal_5753.json <> Reports: 5754
Test Set: Bugzilla_trivial_682.json
File:Bugzilla_trivial_682.json <> Reports: 683
Test Set: Core_blocker_1618.json
File:Core_blocker_1618.json <> Reports: 1619
Test Set: Core_critical_10000.json
File:Core_critical_10000.json <> Reports: 10000
Test Set: Core_major_7523.json
File:Core_major_7523.json <> Reports: 7254
Test Set: Core_minor_3105.json
File:Core_minor_3105.json <> Reports: 3106
Test Set: Core_normal_10000.json
File:Core_normal_10000.json <> Reports: 10000
Test Set: Core_trivial_1484.json
File:Core_trivial_1484.json <> Reports: 1485
Test Set: Firefox_blocker_223.json
Fil

In [147]:
summary=[]
label=[]


for j in bug_reports:
    summary.append(j['summary'])
    
    label.append(j['severity'])
    
print('summary',len(summary))
print('label',len(label))


summary 66427
label 66427


In [148]:
labels=[]
for x in range(len(label)):
    if label[x]=="blocker":
        label[x]=1
    elif label[x]=="critical":
        label[x]=1
    elif label[x]=="major":
        label[x]=1
    elif label[x]=="normal":
        label[x]=0
    elif label[x]=="minor":
        label[x]=0
    elif label[x]=="trivial":
        label[x]=0
 
    
    labels.append(label[x])
print("test_labels:",len(labels))
 

test_labels: 66427


In [166]:
from sklearn.model_selection import train_test_split
X_train, X_test, train_label, test_label = train_test_split(summary, labels, test_size=0.33, random_state=42)

In [180]:
#CountVectorizer
#Transforms text into a sparse matrix of n-gram counts.

#TfidfTransformer
#Performs the TF-IDF transformation from a provided matrix of counts.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict',
                strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None,
                analyzer='word', stop_words=None, ngram_range=(1,2),
                max_df=0.1, min_df=1, max_features=9000, vocabulary=None, binary=False,
                norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [181]:
train1 = vectorizer.fit_transform(X_train)
test1 = vectorizer.transform(X_test)
train_summary=train1.toarray()
test_summary=test1.toarray()

In [182]:
print(train.shape)
print(test.shape)

(44506, 9000)
(21921, 9000)


In [183]:
print(vectorizer.get_feature_names())

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '0a1', '0a1 crash', '0b1', '0rc1', '0x0', '0x00000000', '0x7', '0x80004005', '0x80004005 ns_error_failure', '10', '10 10', '10 and', '10 crash', '100', '100 cpu', '1000', '11', '12', '13', '14', '15', '16', '17', '18', '18 20', '19', '1px', '1st', '20', '20 22', '200', '2000', '2006', '2008', '2009', '2016', '2017', '2017 09', '2018', '2019', '21', '22', '22 and', '23', '24', '25', '256', '26', '27', '28', '29', '2d', '2nd', '2x', '30', '31', '32', '32 bit', '34', '36', '38', '3rd', '3rd party', '40', '404', '45', '49', '50', '500', '52', '55', '56', '57', '59', '60', '64', '64 bit', '68', '70', '72', '75', '8859', '95', '99', '__proto__', '_blank', 'a11y', 'ab', 'ability', 'ability to', 'able', 'able to', 'abort', 'abort message', 'about', 'about blank', 'about cache', 'about config', 'about dialog', 'about firefox', 'about home', 'about license', 'about mozilla', 'about newtab', 'about permissions', 'about plugins', 'about 

In [184]:
print(train.shape)
print(test.shape)

(44506, 9000)
(21921, 9000)


In [185]:
from sklearn import metrics
#1. MUltinomail_naive Bayes
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(train_summary, train_label)
predicted = clf.predict(test_summary)


metrics.confusion_matrix(test_label, predicted)
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(test_label, predicted)
print(average_precision)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_label, predicted)

0.6165104578052315


array([[13217,   993],
       [ 3312,  4399]], dtype=int64)

In [95]:
print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.73      0.99      0.84      3858
           1       0.43      0.01      0.03      1466

    accuracy                           0.72      5324
   macro avg       0.58      0.50      0.43      5324
weighted avg       0.64      0.72      0.62      5324

Accuracy: 0.723328324567994
F1 score: 0.02772277227722772
Recall: 0.01432469304229195
Precision: 0.42857142857142855
matthews_corrcoef: 0.03305801553755331

 confussion matrix:
 [[3830   28]
 [1445   21]]


In [186]:
# 2. Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=2, max_depth=4)
decision_tree = decision_tree.fit(train_summary, train_label)
predicted=decision_tree.predict(test_summary)
from sklearn.metrics import confusion_matrix
confusion_matrix(test_label, predicted)




array([[14028,   182],
       [ 5026,  2685]], dtype=int64)

In [ ]:
print(metrics.classification_report(test_labels, predicted, zero_division=1))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

In [55]:
# 3. SVM
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

X, y = make_classification(n_features=5, random_state=0)
clf = make_pipeline(StandardScaler(with_mean=False),
                     LinearSVC(random_state=0, tol=1e-5))

clf.fit(X1, train_labels)
#print(clf.named_steps['linearsvc'].intercept_)
#print(clf.named_steps['linearsvc'].coef_)
predicted=clf.predict(Y1)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

MemoryError: Unable to allocate 2.96 GiB for an array with shape (40638, 9760) and data type float64

In [56]:
# 4. Bernouli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
clf_bn = BernoulliNB()
clf_bn.fit(X1, train_labels)
predicted=clf_bn.predict(Y1)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

MemoryError: Unable to allocate 2.96 GiB for an array with shape (40638, 9760) and data type float64

In [175]:
#5.KNN 
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()  
model.fit(train, train_labels)

predicted=model.predict(test)

print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.72      1.00      0.83      1279
           1       0.00      0.00      0.00       506

    accuracy                           0.72      1785
   macro avg       0.36      0.50      0.42      1785
weighted avg       0.51      0.72      0.60      1785

Accuracy: 0.7159663865546219
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: -0.014891641110936946

 confussion matrix:
 [[1278    1]
 [ 506    0]]


In [ ]:
# 6. Logistic Regression 

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0).fit(train, train_labels)
predicted = clf_lr.predict(test)


print(metrics.classification_report(test_labels, predicted))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

In [176]:
# 7. Mutlilayer_perceptron
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5 , hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(train, train_labels)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,solver='lbfgs')
predicted=clf.predict(test)

print(metrics.classification_report(test_labels, predicted, zero_division=1))
print ('Accuracy:', accuracy_score(test_labels, predicted))
print ('F1 score:', f1_score(test_labels, predicted))
print ('Recall:', recall_score(test_labels, predicted))
print ('Precision:', precision_score(test_labels, predicted))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels, predicted))
print ('\n confussion matrix:\n',confusion_matrix(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.72      1.00      0.83      1279
           1       0.00      0.00      0.00       506

    accuracy                           0.72      1785
   macro avg       0.36      0.50      0.42      1785
weighted avg       0.51      0.72      0.60      1785

Accuracy: 0.715406162464986
F1 score: 0.0
Recall: 0.0
Precision: 0.0
matthews_corrcoef: -0.021065865763041064

 confussion matrix:
 [[1277    2]
 [ 506    0]]


C:\Users\Ali\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
